In [3]:



import os
import pandas as pd
import numpy as np
from sklearn.metrics import (accuracy_score, f1_score, precision_score, 
                             recall_score, roc_auc_score, mean_squared_error, r2_score)
from sklearn.model_selection import train_test_split
import xgboost as xgb
import math

from config import region_mapping, set1, to_model_a, to_model_c, to_model_ac, to_model

# Define functions


def get_existing_file_path(*paths):
    """Checks and returns the first existing file path from the given list of paths."""
    for path in paths:
        if os.path.exists(path):
            return path
    return None

def invert_region_mapping(mapping):
    """Inverts a region to country mapping to a country to region mapping."""
    return {country: region for region, countries in mapping.items() for country in countries}


def read_and_process_data(file_path, columns, region_mapping):
    """Reads data from CSV, maps countries to regions, and selects specified columns."""
    try:
        df = pd.read_csv(file_path, low_memory=False)
        print(f"Data loaded. Shape: {df.shape}")

        country_to_region = invert_region_mapping(region_mapping)
        df['Region'] = df['Country'].map(country_to_region)

        df_processed = df[columns].copy()
        df_processed.loc[:, 'Processed food in diet'] = df_processed['Processed food in diet'].replace({
            'Rarely/never': 'Rarely/Never',
            'A few times in a day': 'Several times a day',
            'Several days a week': 'A few times a week',
            'Many times in a day': 'Several times a day',
            'At least once a day': 'Several times a day'
        })

        return df_processed
    except Exception as e:
        print(f"An error occurred: {e}")
        return None




# Define file paths
path = '/Users/jerzybala/Desktop/Simulation Experiments Aug-Sep 2023/PROCESSED_FOOD_Dec2023.csv'
path2 = '/Volumes/Desktop/Simulation Experiments Aug-Sep 2023/PROCESSED_FOOD_Dec2023.csv'

# Determine which file path exists
file_path = get_existing_file_path(path, path2)

# Process the data if file exists
if file_path:
    df_org2 = read_and_process_data(file_path, set1, region_mapping)
    print(f"Processed DataFrame shape: {df_org2.shape}\n")
else:
    print("File not found in the given paths.")





Data loaded. Shape: (401682, 239)
Processed DataFrame shape: (401682, 51)



## Transitions


In [17]:
import pandas as pd

def update_food_category_based_on_modes(df, source_cat, target_cat, features):
    """
    Updates 'Processed food in diet' category in the DataFrame based on mode matching.
    
    :param df: DataFrame to be processed.
    :param source_cat: The category to analyze for mode (e.g., 'Rarely/Never').
    :param target_cat: The category to update to (e.g., 'Once a day').
    :param features: List of features to calculate mode and match against.
    :return: DataFrame with updated 'Processed food in diet' values.
    """
    def calculate_modes(dataframe, category, feature_list):
        category_df = dataframe[dataframe['Processed food in diet'] == category]
        return {feature: category_df[feature].mode()[0] for feature in feature_list}

    def matches_modes(row, modes):
        return sum(row[feature] == mode for feature, mode in modes.items()) >= 3

    df_copy = df.copy()

    # Calculate mode for the specified category
    modes = calculate_modes(df_copy, source_cat, features)

    # Identify rows in target category that match the mode of source category
    matching_condition = df_copy[df_copy['Processed food in diet'] == target_cat].apply(lambda row: matches_modes(row, modes), axis=1)
    matching_rows_indices = matching_condition[matching_condition].index

    # Modify 'Processed food in diet' for these rows in df_copy
    df_copy.loc[matching_rows_indices, 'Processed food in diet'] = source_cat

    return df_copy

# Usage example
updated_df = update_food_category_based_on_modes(df_org2, 'Rarely/Never', 'Once a day', 
                                                 ['Frequency of getting a good nights sleep', 
                                                  'Frequency of doing exercise', 
                                                  'Frequency of Socializing'])

# Check the modified distribution
print(updated_df['Processed food in diet'].value_counts())









Rarely/Never           154981
A few times a month    125715
A few times a week      85120
Once a day              19340
Several times a day     16526
Name: Processed food in diet, dtype: int64


In [ ]:
print(df_org2['Processed food in diet'].value_counts())



In [18]:
import pandas as pd

def update_food_category_based_on_modes(df, source_cat, target_cat, features):
    """
    Updates 'Processed food in diet' category in the DataFrame based on mode matching.
    
    :param df: DataFrame to be processed.
    :param source_cat: The category to analyze for mode (e.g., 'Rarely/Never').
    :param target_cat: The category to update to (e.g., 'Once a day').
    :param features: List of features to calculate mode and match against.
    :return: DataFrame with updated 'Processed food in diet' values.
    """
    def calculate_modes(dataframe, category, feature_list):
        category_df = dataframe[dataframe['Processed food in diet'] == category]
        return {feature: category_df[feature].mode()[0] for feature in feature_list}

    def matches_modes(row, modes):
        return sum(row[feature] == mode for feature, mode in modes.items()) >= 3

    df_copy = df.copy()

    # Calculate mode for the specified category
    modes = calculate_modes(df_copy, source_cat, features)

    # Identify rows in target category that match the mode of source category
    matching_condition = df_copy[df_copy['Processed food in diet'] == target_cat].apply(lambda row: matches_modes(row, modes), axis=1)
    matching_rows_indices = matching_condition[matching_condition].index

    # Modify 'Processed food in diet' for these rows in df_copy
    df_copy.loc[matching_rows_indices, 'Processed food in diet'] = source_cat

    return df_copy

# Usage example
updated_df = update_food_category_based_on_modes(df_org2, 'Rarely/Never', 'Once a day', 
                                                 ['Frequency of getting a good nights sleep', 
                                                  'Frequency of doing exercise', 
                                                  'Frequency of Socializing'])

# Check the modified distribution
print(updated_df['Processed food in diet'].value_counts())



Rarely/Never           154981
A few times a month    125715
A few times a week      85120
Once a day              19340
Several times a day     16526
Name: Processed food in diet, dtype: int64


# Scenario processing

## Age

In [4]:
df=df_org2.copy()
#df = df3

#Age filter
ages = ['18-24', '21-24', '18', '19', '20']
df = df[df['Age'].isin(ages)]

# Select only the columns with object type (commonly used for categorical features)
categorical_features = df.select_dtypes(include=['object']).columns
# Apply pd.get_dummies to the categorical features with a prefix
encoded_features = pd.get_dummies(df[categorical_features])
# Concatenate the encoded features with the original DataFrame (excluding the original categorical features)

df = pd.concat([df.drop(columns=categorical_features), encoded_features], axis=1)

df.columns.tolist()



df = df[to_model_a].copy()
print(len(df))



71506


# Countries

In [19]:
#df_c=df_org2.copy()
df_c = updated_df

num_empty_spaces = df_c['Processed food in diet'].isna().sum()
print("num_empty_spaces;",num_empty_spaces)


# Check the modified distribution
print(df_c['Processed food in diet'].value_counts())

# Country filter
countries = ['United States','United Kingdom','Australia']
df_c = df_c[df_c['Country'].isin(countries)]
#print(df_c['Country'].value_counts())

df_c = df_c.drop(columns='Country', axis=1)
#print(df_c['Country'].value_counts())


# Check the modified distribution
print(df_c['Processed food in diet'].value_counts())

#print(df['Country'].value_counts())

# Select only the columns with object type (commonly used for categorical features)
categorical_features = df_c.select_dtypes(include=['object']).columns
# Apply pd.get_dummies to the categorical features with a prefix
encoded_features = pd.get_dummies(df_c[categorical_features])
# Concatenate the encoded features with the original DataFrame (excluding the original categorical features)

df_c = pd.concat([df_c.drop(columns=categorical_features), encoded_features], axis=1)



df_c=df_c[to_model_c].copy()
print(len(df_c))



num_empty_spaces; 0
Rarely/Never           154981
A few times a month    125715
A few times a week      85120
Once a day              19340
Several times a day     16526
Name: Processed food in diet, dtype: int64
A few times a week     11113
A few times a month     8436
Rarely/Never            5209
Once a day              3451
Several times a day     2570
Name: Processed food in diet, dtype: int64
30779


## Countries and Age

In [ ]:
df=df_org2.copy()

#Age filter
ages = ['18-24', '21-24', '18', '19', '20']
df = df[df['Age'].isin(ages)]

# Country filter
countries = ['United States','United Kingdom','Australia']
df = df[df['Country'].isin(countries)]
#print(df_c['Country'].value_counts())

print(df['Country'].value_counts())


print(len(df))

# Check the modified distribution
print(df['Processed food in diet'].value_counts())


# Select only the columns with object type (commonly used for categorical features)
categorical_features = df.select_dtypes(include=['object']).columns
# Apply pd.get_dummies to the categorical features with a prefix
encoded_features = pd.get_dummies(df[categorical_features])
# Concatenate the encoded features with the original DataFrame (excluding the original categorical features)

df = pd.concat([df.drop(columns=categorical_features), encoded_features], axis=1)




df=df[to_model_ac].copy()
print(len(df))


In [ ]:
#df.columns.tolist()

In [ ]:
df=df_org2.copy()

# Select only the columns with object type (commonly used for categorical features)
categorical_features = df.select_dtypes(include=['object']).columns
# Apply pd.get_dummies to the categorical features with a prefix
encoded_features = pd.get_dummies(df[categorical_features])
# Concatenate the encoded features with the original DataFrame (excluding the original categorical features)

df = pd.concat([df.drop(columns=categorical_features), encoded_features], axis=1)




df=df[to_model].copy()
print(len(df))





In [20]:
df=df_c

In [21]:



X = df.drop(columns=['MHQ_Sign','Overall MHQ'], axis=1)

# classification
y = df['MHQ_Sign']
# regressioin
y_regression = df['Overall MHQ']

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=0.3, random_state=42)

In [ ]:
X.to_csv("/Users/jerzybala/Desktop/data_causal_all_UPF_Age_18-24 and Anglosaxon_behavior_change.csv")

In [ ]:
X.columns.tolist()

In [ ]:
X['UPF_Social1'] = X['Processed food in diet_Rarely/Never'] + X['Frequency of Socializing_Once a week'] + X['Frequency of Socializing_Several days a week']


In [26]:
par1 = {
    'n_estimators': 200, 
    'learning_rate': 0.01,
    'max_depth': 10, 
    'min_child_weight': 1, 
    'gamma': 0.01, 
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Initialize XGBoost classifier and train

model_C = xgb.XGBClassifier(**par1)
model_C.fit(X_train, y_train)

# Predict and evaluate
predictions = model_C.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
auc = roc_auc_score(y_test, model_C.predict_proba(X_test)[:, 1])

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"AUC: {auc}")


Accuracy: 0.8121074290664934
F1 Score: 0.8863264102732097
Precision: 0.8264907135874878
Recall: 0.9555021895747987
AUC: 0.839543874909908


In [ ]:
from sklearn.model_selection import GridSearchCV

# gs = GridSearchCV(XGBClassifier(), 
#                   param_grid, 
#                   scoring='accuracy',
#                   cv=5)
                  
# gs.fit(X_train, y_train)


# print(gs.best_params_)
# print(gs.best_score_)

In [ ]:
# Initialize XGBoost regressor and train
#model = xgb.XGBRegressor()
# X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=0.3, random_state=42)

from sklearn.metrics import mean_absolute_error
#model_R = xgb.XGBRegressor()

model_R = xgb.XGBRegressor(
n_estimators=500,
learning_rate=0.1,
max_depth=6,
min_child_weight=1,
gamma=0.2,
subsample=0.4,
colsample_bytree=0.8,
#reg_alpha=10,
reg_lambda=0.1
)

model_R.fit(X_train_reg, y_train_reg)

# Separate predictions based on the sign of y_test
predictions = model_R.predict(X_test_reg)

mse = mean_squared_error(y_test_reg, predictions)
rmse = math.sqrt(mse)
r2 = r2_score(y_test_reg, predictions)
mae = mean_absolute_error(y_test_reg, predictions)

print('mae:', mae)
print('rmse:',rmse)
print('r2:', r2)


In [15]:
import shap

baseline_shap_values = shap.TreeExplainer(model_C).shap_values(X)

[15:11:29] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


In [27]:
variant1_shap_values = shap.TreeExplainer(model_C).shap_values(X)

[15:18:27] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


In [ ]:
import shap

explainer_C = shap.Explainer(model_C)
shap_values_C = explainer_C(X_train)

In [ ]:

shap_values_C_df = pd.DataFrame(shap_values_C.values, columns=shap_values_C.feature_names)
shap_values_C_df.head(4)


In [28]:
baseline_ranking = np.abs(baseline_shap_values).mean(0).argsort()
variant1_ranking = np.abs(variant1_shap_values).mean(0).argsort()
print(baseline_ranking == variant1_ranking)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
 False  True False False False False  True  True  True False False  True
  True  True False  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True]


In [ ]:
import shap
import matplotlib.pyplot as plt

# Specify the plot size directly in the SHAP summary_plot function
shap.summary_plot(shap_values_C, X_train, show=False, plot_size=(16,10), max_display=50)  # Adjust the size (width, height) as needed

plt.title(f'SHAP Summary Plot for Classification Age=18-24 and Core Anglosphere > UPF more Raerly')
plt.show()

In [29]:
modified_feature = 0 
orig_shap = np.abs(baseline_shap_values[:, modified_feature]).mean()
new_shap = np.abs(variant1_shap_values[:, modified_feature]).mean()
print(f"{modified_feature} SHAP value changed by {new_shap - orig_shap}")


0 SHAP value changed by 7.053371518850327e-05


In [ ]:
# visualize the training set predictions
#  takes too long to run

import numpy as np

# Assuming shap_values_C is a numpy array
sampled_indices = np.random.choice(shap_values_C.shape[0], size=20000, replace=False)
sampled_shap_values_C = shap_values_C[sampled_indices]

shap.plots.force(sampled_shap_values_C)

In [ ]:
# create a SHAP dependence plot to show the effect of a single feature across the whole dataset
shap.dependence_plot('Processed food in diet_Rarely/Never', shap_values_C.values, X_train, interaction_index="Frequency of Socializing_Rarely/Never")


In [ ]:
# summarize the effects of all the features
shap.plots.beeswarm(shap_values_C,max_display=15)

In [ ]:
import shap
from shap import Explainer

shap.force_plot(
    shap.Explainer.expected_value[1], shap_values_C[1][:1000, :], X_train.iloc[:1000, :]
)

In [ ]:
shap.plots.waterfall(shap_values[X_train])


In [ ]:
explainer_R = shap.Explainer(model_R)
shap_values_R = explainer_R(X_train_reg)


In [ ]:
import shap

# Load SHAP values from each model
baseline_shap_values = shap.TreeExplainer(baseline_model).shap_values(X)
variant1_shap_values = shap.TreeExplainer(variant1_model).shap_values(X)
variant2_shap_values = shap.TreeExplainer(variant2_model).shap_values(X)

# Step 5 analysis

# Summary plot 
shap.summary_plot(baseline_shap_values, X)
shap.summary_plot(variant1_shap_values, X)

# Compare feature ranking
baseline_ranking = np.abs(baseline_shap_values).mean(0).argsort()
variant1_ranking = np.abs(variant1_shap_values).mean(0).argsort()
print(baseline_ranking == variant1_ranking)

# Magnitude difference 
modified_feature = 0 
orig_shap = np.abs(baseline_shap_values[:, modified_feature]).mean()
new_shap = np.abs(variant1_shap_values[:, modified_feature]).mean()
print(f"{modified_feature} SHAP value changed by {new_shap - orig_shap}")

# Step 6 analysis
corr_matrix = np.corrcoef(baseline_shap_values, rowvar=False) 
mod_corr_matrix = np.corrcoef(variant1_shap_values, rowvar=False)
changed_corrs = np.nonzero(np.abs(corr_matrix - mod_corr_matrix) > threshold) 
print(f"Modifying feature {modified_feature} affected dependencies with: {changed_corrs}")

In [ ]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
import numpy as np

# Assuming 'df' is your DataFrame with X and Y
# Replace 'df' with the name of your DataFrame
# Ensure your target column is named 'target'

def calculate_information_gain(X,y):
  
    # Calculating mutual information
    mutual_info = mutual_info_classif(X, y)

    # Creating a DataFrame for easier visualization
    info_gain_df = pd.DataFrame(mutual_info, index=X.columns, columns=['Information Gain'])
    
    # Sorting the DataFrame based on information gain
    sorted_info_gain = info_gain_df.sort_values(by='Information Gain', ascending=False)

    return sorted_info_gain

#Example usage:
sorted_info_gain = calculate_information_gain(X,y)
print(sorted_info_gain)


In [ ]:
from tabulate import tabulate

#Example usage:
sorted_info_gain = calculate_information_gain(X,y)
print(tabulate(sorted_info_gain, headers='keys', tablefmt='psql'))
